In [1]:
from local_tree_decomp import *
from local_hom_count import *
from local_hom_count_int import *
from local_helpers import *
from local_hom_count_int_pre import *
from local_hom_count_int_pre_np import *
from local_hom_count_int_pre_adj import *

In [2]:
def speed_test(graph, target_graph, naive=True):
    basic_result = count_homomorphisms(graph, target_graph)
    int_rep_result = count_homomorphisms_int(graph, target_graph)
    better_int_rep_result = count_homomorphisms_int_pre(graph, target_graph)

    results = [basic_result, int_rep_result, better_int_rep_result]

    if naive:
        brute_force_result = len(enumerate_homomorphisms(graph, target_graph))
        results.append(brute_force_result)

    if all(result == better_int_rep_result for result in results):
        print(target_graph.order())
        # print(better_int_rep_result)

        if naive:
            print('\nBrute force:')
            %timeit len(enumerate_homomorphisms(graph, target_graph))
        
        print('\nBasic:')
        %timeit count_homomorphisms(graph, target_graph)
        
        print('\nInt representation:')
        %timeit count_homomorphisms_int(graph, target_graph)
    
        print('\nBetter int representation:')
        %timeit count_homomorphisms_int_pre(graph, target_graph)


In [3]:
# K3 = graphs.CompleteGraph(3)
# K1_4 = graphs.CompleteBipartiteGraph(1, 4)
# C4 = graphs.CycleGraph(4)

# for i in range(1, 16):
#     target = graphs.CompleteGraph(i * 10)
#     speed_test(K3, target)


In [4]:
graph = graphs.CompleteBipartiteGraph(1, 4)
target_graph = graphs.CompleteGraph(10)

%timeit count_homomorphisms_int_pre(graph, target_graph)
%timeit count_homomorphisms_int_pre_np(graph, target_graph)
%timeit count_homomorphisms_int_pre_adj(graph, target_graph)

5.99 ms ± 271 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.67 ms ± 433 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
26.3 ms ± 1.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
graph = graphs.CompleteBipartiteGraph(2, 4)
target_graph = graphs.CompleteGraph(20)

%prun enumerate_homomorphisms(graph, target_graph)

KeyboardInterrupt: 

In [7]:
%load_ext line_profiler

from local_hom_count_int_pre import _add_intro_node_int_pre, is_valid_mapping

graph = graphs.CompleteBipartiteGraph(2, 4)
target_graph = graphs.RandomGNP(30, 0.5, seed=42)

%lprun -s -u 1e-03 -f is_valid_mapping count_homomorphisms_int_pre(graph, target_graph)

Timer unit: 0.001 s

Total time: 0.800064 s
File: /Users/Latisp/github/local-hom-count/local_hom_count_int_pre.py
Function: is_valid_mapping at line 111

Line #      Hits         Time  Per Hit   % Time  Line Contents
   111                                           def is_valid_mapping(mapped_intro_vtx, mapped_nbhrs, target_graph):
   112    215038        163.5      0.0     20.4      for vtx in mapped_nbhrs:
   113    173160        532.5      0.0     66.6          if not target_graph.has_edge(mapped_intro_vtx, vtx):
   114     94982         78.7      0.0      9.8              return False
   115                                           
   116     41878         25.5      0.0      3.2      return True

In [8]:
from local_hom_count_int_pre_adj import _add_intro_node_int_pre_adj, is_valid_mapping

graph = graphs.CompleteBipartiteGraph(2, 4)
target_graph = graphs.RandomGNP(30, 0.5, seed=42)

%lprun -s -u 1e-03 -f is_valid_mapping count_homomorphisms_int_pre_adj(graph, target_graph)

Timer unit: 0.001 s

Total time: 5.67367 s
File: /Users/Latisp/github/local-hom-count/local_hom_count_int_pre_adj.py
Function: is_valid_mapping at line 111

Line #      Hits         Time  Per Hit   % Time  Line Contents
   111                                           def is_valid_mapping(mapped_intro_vtx, mapped_nbhrs, target_graph_matrix):
   112                                               # current_row = target_graph_matrix[mapped_intro_vtx]
   113    215038        177.3      0.0      3.1      for vtx in mapped_nbhrs:
   114    173160       5379.1      0.0     94.8          if not target_graph_matrix[mapped_intro_vtx][vtx]:
   115     94982         89.2      0.0      1.6              return False
   116                                           
   117     41878         28.1      0.0      0.5      return True

In [57]:
print(count_homomorphisms_int_pre(graph, target_graph))
print(count_homomorphisms_int_pre_adj(graph, target_graph))

2998736
2998736
